In [166]:
import requests
from user_agent import generate_user_agent
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [167]:
HEADERS = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
URL = 'http://notelections.online'
LINK = 'http://notelections.online/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'

### Анализ ссылок сайта
http://notelections.online/region/izbirkom?action=tvdTree&tvdchildren=true&vrn=100100084849062&tvd=100100084849067

<i>последняя:</i>
http://notelections.online/region/izbirkom?action=show&root=1000001&tvd=2012000364363&vrn=100100084849062&prver=0&pronetvd=null&region=1&sub_region=1&type=227&report_mode=null

<i>первая в след разделе:</i>
http://notelections.online/region/izbirkom?action=show&root=1000002&tvd=2042000367645&vrn=100100084849062&prver=0&pronetvd=null&region=2&sub_region=2&type=227&report_mode=null

<i>рандомная:</i>
http://notelections.online/region/izbirkom?action=show&root=1000082&tvd=2412000447687&vrn=100100084849062&prver=0&pronetvd=null&region=91&sub_region=91&type=227&report_mode=null

### С помощью анализа сайта разработан план скрапинга данных
1. Начальная ссылка: http://notelections.online/region/izbirkom?action=show&root=0&tvd=100100084849066&vrn=100100084849062&prver=0&pronetvd=null&region=0&sub_region=0&type=227&report_mode=null
2. a.tree-open -> ul li.tree-li (по каждому, запоминаем id тега) -> a:nth-chield(2) href
3. Переходим по полученной ссылке и ищем по id элемент -> ul -> li.tree-li (по каждому) -> a:nth-chield(2) href
4. Переходим по полученной ссылке и берем таблицу: .table-fixed-columns.table-bordered.table-striped.table-sm
5. Выделение данных из таблицы: thead (по всем) -> tр (начиная с 3 его) - соберем заголовки, и tr (по всем) -> td (начиная с 3 его) - соберем числовые данные

In [170]:
page_response = requests.get(LINK, timeout=5, headers=HEADERS)
columns = ['Регион', 'Тик', 'УИК']

if page_response.status_code == 200:
    soup = BeautifulSoup(page_response.text, 'html.parser')
    
    # Список всех регионов
    li_table = soup.find('table', class_='table-fixed-columns')
    li_list = li_table.findAll('th')[3:]

    # Список всех стандартных полей из протокола
    columns_list = li_table.find('tbody').findAll('tr')
    
    for column in columns_list:
        columns.append(column.findAll('td')[1].text)

    df = pd.DataFrame(columns=columns)

    for li in tqdm(li_list):
        link = li.find('a').attrs['href']
        region_page = requests.get(URL + link, timeout=10, headers=HEADERS)

        if region_page.status_code == 200:
            region_soup = BeautifulSoup(region_page.text, 'html.parser')

            # Список всех подрегионов
            li_list_2 = region_soup.find('table', class_='table-fixed-columns').findAll('th')[3:]

            for li_2 in li_list_2:
                link_tik = li_2.find('a')
                href = link_tik.attrs['href']

                tik_page = requests.get(URL + href, timeout=10, headers=HEADERS)

                if tik_page.status_code == 200:
                    tik_soup = BeautifulSoup(tik_page.text, 'html.parser')

                    # with open("link.html", 'w', encoding='utf-8') as file:
                    #     file.write(tik_page.text)

                    table = tik_soup.find('table', class_='table-fixed-columns')

                    # список всех уиков:
                    head = table.findAll('th')[2:]
                    col_count = len(head)

                    rows = table.find('tbody').findAll('tr')
                    row_count = len(rows)

                    data = [[] * row_count for i in range(col_count)]

                    for row in rows:
                        td = row.findAll('td')[2:]
                        
                        for t in range(len(td)):
                            text = td[t].find('nobr').text
                            data[t].append(text)


                    for data_row in data:
                        append_row = pd.Series([li.text, link_tik.text, head[data.index(data_row)].text, data_row[0], data_row[1], data_row[2], data_row[3], data_row[4], data_row[5], data_row[6], data_row[7], data_row[8], data_row[9], data_row[10], data_row[11], data_row[12], data_row[13], data_row[14], data_row[15], data_row[16], data_row[17], data_row[18], data_row[19]], index=df.columns)
                        df = df.append(append_row, ignore_index=True)
                
                else:
                    print("не удалось взять данные из " + str(URL) + str(href))
                    print("li_2 " + str(li_2))

        else:
            print("не удалось взять данные из " + str(URL) + str(link))
            print("li " + str(li))


else:
    print("page response status code is " + str(page_response.status_code) + " on page link: " + str(LINK))

 98%|█████████▊| 85/87 [2:36:24<03:40, 110.41s/it]


AttributeError: 'NoneType' object has no attribute 'attrs'

Упало на 85/87 регионе. Оказалось, что там нет подрегионов (ТИК). Соберем li_list[85] до li_list[86] отдельно.

In [171]:
df.to_csv('result.csv')

In [184]:
print(li_list[85])

<th class="text-center"><nobr><a href="/region/region/izbirkom?action=show&amp;tvd=100100084849066&amp;vrn=100100084849062&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100084849218&amp;type=227" style="text-decoration: none">Город Байконур (Республика Казахстан)</a></nobr></th>


In [191]:
df_add = pd.DataFrame(columns=columns)

for li in tqdm(li_list[85:]):
    link = li.find('a').attrs['href']
    region_page = requests.get(URL + link, timeout=10, headers=HEADERS)

    if region_page.status_code == 200:
        region_soup = BeautifulSoup(region_page.text, 'html.parser')

        table = region_soup.find('table', class_='table-fixed-columns')

        # список всех уиков:
        head = table.findAll('th')[2:]
        col_count = len(head)

        rows = table.find('tbody').findAll('tr')
        row_count = len(rows)

        data = [[] * row_count for i in range(col_count)]

        for row in rows:
            td = row.findAll('td')[2:]
            
            for t in range(len(td)):
                text = td[t].find('nobr').text
                data[t].append(text)


        for data_row in data:
            append_row = pd.Series([li.text, None, head[data.index(data_row)].text, data_row[0], data_row[1], data_row[2], data_row[3], data_row[4], data_row[5], data_row[6], data_row[7], data_row[8], data_row[9], data_row[10], data_row[11], data_row[12], data_row[13], data_row[14], data_row[15], data_row[16], data_row[17], data_row[18], data_row[19]], index=df_add.columns)
            df_add = df_add.append(append_row, ignore_index=True)

100%|██████████| 2/2 [00:16<00:00,  8.10s/it]


In [187]:
df = df.append(df_add)
df.to_csv('result.csv')

In [188]:
df

,Регион,Тик,УИК,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,Республика Адыгея (Адыгея),Адыгейская,Сумма,11624,11350,0,10833,399,118,399,...,0,0,24,655,149,10229,62,16,15,25
1,Республика Адыгея (Адыгея),Адыгейская,УИК №1,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
2,Республика Адыгея (Адыгея),Адыгейская,УИК №2,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
3,Республика Адыгея (Адыгея),Адыгейская,УИК №3,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
4,Республика Адыгея (Адыгея),Адыгейская,УИК №4,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Территория за пределами РФ,None,УИК №1310,2892,2900,1000,1891,0,9,1000,...,0,0,0,3,10,2878,0,0,0,0
5996,Территория за пределами РФ,None,УИК №1311,2867,2900,921,1941,0,38,921,...,0,0,0,3,5,2853,1,0,0,0
5997,Территория за пределами РФ,None,УИК №1312,2895,2900,927,1964,0,9,927,...,0,0,1,3,6,2876,2,1,1,1
5998,Территория за пределами РФ,None,УИК №1313,3397,3450,1103,2281,0,66,1103,...,0,0,0,7,24,3353,0,0,0,0
